In [2]:
# imports and stuff
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

The Data I chose: Spotify Music Dataset from kaggle ( https://www.kaggle.com/datasets/solomonameh/spotify-music-dataset )
It consits of two csv files; high popularity and low popularity

In [3]:
# Importing the data
high = pd.read_csv('../data/high_popularity_spotify_data.csv')
low = pd.read_csv('../data/low_popularity_spotify_data.csv')
# Could I create a merged dataset with an extra 'popularity' column to distinguish the two?

In [4]:
print('high', high.columns)
# print('high', low.columns)

print('high', high.shape)
print('low', high.shape)

high Index(['energy', 'tempo', 'danceability', 'playlist_genre', 'loudness',
       'liveness', 'valence', 'track_artist', 'time_signature', 'speechiness',
       'track_popularity', 'track_href', 'uri', 'track_album_name',
       'playlist_name', 'analysis_url', 'track_id', 'track_name',
       'track_album_release_date', 'instrumentalness', 'track_album_id',
       'mode', 'key', 'duration_ms', 'acousticness', 'id', 'playlist_subgenre',
       'type', 'playlist_id'],
      dtype='object')
high (1686, 29)
low (1686, 29)


Both have the same columns (and even the same amount of samples)
As they already have a 'track_popularity' column, I might not actually need to create one to distinguish the two,
if I know the threshold for high & low priority.\
Of course, the question arises, how chose this threshold and why. Maybe looking into the domain of this column can tell me more.\
I think a first interesting research question could be:"What parameters influence (the most), whether a song is popular or not

In [5]:
print('high max popularity level', high['track_popularity'].max())
print('high min popularity level', high['track_popularity'].min())
print('low max popularity level', low['track_popularity'].max())
print('low min popularity level' , low['track_popularity'].min())

high max popularity level 100
high min popularity level 68
low max popularity level 68
low min popularity level 11


Looks like the domain is 0-100, i.e. a percentage value and the threshold for low/high popularity seems to be 68.
Maybe, as both datasets have the same amount of observations, it's just the median of the whole dataset\
Let's look at the 5 most popular and unpopular songs of the datasets

In [15]:
high = high.sort_values(by='track_popularity', ascending=False)
print("Most popular:\n", high[["track_name", "track_id", "track_artist", "track_album_name", "track_popularity", "playlist_name"]].head(5))

Most popular:
            track_name                track_id           track_artist  \
676  Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
455  Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
0    Die With A Smile  2plbrEY59IikOBgBGLjaoe  Lady Gaga, Bruno Mars   
677              APT.  5vNRhkKd0yEAg8suGBpjeY       ROSÉ, Bruno Mars   
4                APT.  5vNRhkKd0yEAg8suGBpjeY       ROSÉ, Bruno Mars   

     track_album_name  track_popularity      playlist_name  
676  Die With A Smile               100      Global Top 50  
455  Die With A Smile               100  Top Gaming Tracks  
0    Die With A Smile               100   Today's Top Hits  
677              APT.                98      Global Top 50  
4                APT.                98   Today's Top Hits  


No Taylor Swift, what a nice surprise. I'm guessing this data is from when "Die with a smile" just came out.
I already see some problems with the dataset: Songs are saved multiple times (not sure if they are different versions, or just the same versions e.g. as single and as part of an album) However, as title is saved here, it should be no problem removing duplicates.

In [7]:
display(high[:5])

,energy,tempo,danceability,playlist_genre,loudness,liveness,valence,track_artist,time_signature,speechiness,...,instrumentalness,track_album_id,mode,key,duration_ms,acousticness,id,playlist_subgenre,type,playlist_id
676,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,global,audio_features,37i9dQZEVXbMDoHDwVN2tF
455,0.592,157.969,0.521,gaming,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,modern,audio_features,37i9dQZF1DWTyiBJ6yEqeu
0,0.592,157.969,0.521,pop,-7.777,0.122,0.535,"Lady Gaga, Bruno Mars",3,0.0304,...,0.0,10FLjwfpbxLmW8c25Xyc2N,0,6,251668,0.3080,2plbrEY59IikOBgBGLjaoe,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M
677,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,global,audio_features,37i9dQZEVXbMDoHDwVN2tF
4,0.783,149.027,0.777,pop,-4.477,0.355,0.939,"ROSÉ, Bruno Mars",4,0.2600,...,0.0,2IYQwwgxgOIn7t3iF6ufFD,0,0,169917,0.0283,5vNRhkKd0yEAg8suGBpjeY,mainstream,audio_features,37i9dQZF1DXcBWIGoYBM5M


In [8]:
low = low.sort_values(by='track_popularity', ascending=False)
print("Most unpopular:\n", low[["track_name", "track_id", "track_artist", "track_album_name", "track_popularity"]].tail(5))

Most unpopular:
                   track_name                track_id   track_artist  \
3009          Hit The Ground  1we3IyLdDkmtZctE9xbFgm  Lilly Ahlberg   
2274      A Vaibele A Tsnien  03bEE0edmcuf9iraKHlENO    Dave Tarras   
3081                 Make It  19bfMEAOJGHEAL2pS3gNoS  Daniel Schulz   
3128  Blue Flag / Fun Things  6VvRwqQpgWU1WclXgCOKI8  Jesper Jenset   
3102            It Ain't You  4egXSi9VmYuJaq6Xa1ECQm      Hedda Mae   

                                track_album_name  track_popularity  
3009                              Hit The Ground                11  
2274  Yiddish-American Klezmer Music - 1925-1956                11  
3081                                     Make It                11  
3128                      Blue Flag / Fun Things                11  
3102                         This Might Get Loud                11  


## Cleaning and combining the 2 datasets
As I know the "Threshold" by which a song is classified as high/low popular I can just merge the two datasets. (however I still wonder how this values came to be)\
I also want to remove duplicates. Above I can see that in this case the 3 duplicates of "Die With A Smile" actually have the same track_id.
I do however wonder, if there may also be duplicates with just the same names (i.e. different versions of the song)\
I'm not quite sure yet, how I want to handle that. But first I want to test if there are any such cases anyway

1. Combining the datasets

In [9]:
print(high.columns, "\n", low.columns) # Looking manually they seem to have the same columns, but for some reason in different order
print(sorted(high.columns) == sorted(low.columns)) # Sorting them to compare their column names elementwise

Index(['energy', 'tempo', 'danceability', 'playlist_genre', 'loudness',
       'liveness', 'valence', 'track_artist', 'time_signature', 'speechiness',
       'track_popularity', 'track_href', 'uri', 'track_album_name',
       'playlist_name', 'analysis_url', 'track_id', 'track_name',
       'track_album_release_date', 'instrumentalness', 'track_album_id',
       'mode', 'key', 'duration_ms', 'acousticness', 'id', 'playlist_subgenre',
       'type', 'playlist_id'],
      dtype='object') 
 Index(['time_signature', 'track_popularity', 'speechiness', 'danceability',
       'playlist_name', 'track_artist', 'duration_ms', 'energy',
       'playlist_genre', 'playlist_subgenre', 'track_href', 'track_name',
       'mode', 'uri', 'type', 'track_album_release_date', 'analysis_url', 'id',
       'instrumentalness', 'track_album_id', 'playlist_id', 'track_id',
       'valence', 'key', 'tempo', 'loudness', 'acousticness', 'liveness',
       'track_album_name'],
      dtype='object')
True


As their equality check returned true, we can just go ahead and combine the two

In [10]:
songs = pd.concat([low, high]).sort_values(by='track_popularity', ascending=False)
print(songs.head(5)["track_popularity"], songs.tail(5)["track_popularity"]) # Check if correctly concatenated & sorted

676    100
0      100
455    100
4       98
677     98
Name: track_popularity, dtype: int64 3128    11
2626    11
2483    11
462     11
2274    11
Name: track_popularity, dtype: int64


2. Check for duplicates

In [33]:
idDupes = songs.duplicated(subset=['track_id'], keep=False)
nameDupes = songs.duplicated(subset=['track_name', 'track_artist'], keep=False)


print(
    "Duplicates by track_id:",
    idDupes.sum()
)
print(
    "Duplicates by title and artist:",
    nameDupes.sum()
)

Duplicates by track_id: 631
Duplicates by title and artist: 671


Another problem I noticed was that the dataset doesn't save the genre of the song, but of the playlist it's taken from\
$\Rightarrow$ Removing duplicates from different playlist would remove genres of the song (i.e. missing value 'track_genre'... Why did they not include that?)\
What genre to choose? e.g. 'die with a smile' in gaming playlist with 'gaming' genre... does not really fit...
Maybe do a majority vote on all dupes for the main genre and put all other subgenre array?

In [36]:
display(
    songs[idDupes]
)

,time_signature,track_popularity,speechiness,danceability,playlist_name,track_artist,duration_ms,energy,playlist_genre,playlist_subgenre,...,track_album_id,playlist_id,track_id,valence,key,tempo,loudness,acousticness,liveness,track_album_name
676,3.0,100,0.0304,0.521,Global Top 50,"Lady Gaga, Bruno Mars",251668.0,0.592,pop,global,...,10FLjwfpbxLmW8c25Xyc2N,37i9dQZEVXbMDoHDwVN2tF,2plbrEY59IikOBgBGLjaoe,0.535,6.0,157.969,-7.777,0.3080,0.1220,Die With A Smile
0,3.0,100,0.0304,0.521,Today's Top Hits,"Lady Gaga, Bruno Mars",251668.0,0.592,pop,mainstream,...,10FLjwfpbxLmW8c25Xyc2N,37i9dQZF1DXcBWIGoYBM5M,2plbrEY59IikOBgBGLjaoe,0.535,6.0,157.969,-7.777,0.3080,0.1220,Die With A Smile
455,3.0,100,0.0304,0.521,Top Gaming Tracks,"Lady Gaga, Bruno Mars",251668.0,0.592,gaming,modern,...,10FLjwfpbxLmW8c25Xyc2N,37i9dQZF1DWTyiBJ6yEqeu,2plbrEY59IikOBgBGLjaoe,0.535,6.0,157.969,-7.777,0.3080,0.1220,Die With A Smile
4,4.0,98,0.2600,0.777,Today's Top Hits,"ROSÉ, Bruno Mars",169917.0,0.783,pop,mainstream,...,2IYQwwgxgOIn7t3iF6ufFD,37i9dQZF1DXcBWIGoYBM5M,5vNRhkKd0yEAg8suGBpjeY,0.939,0.0,149.027,-4.477,0.0283,0.3550,APT.
677,4.0,98,0.2600,0.777,Global Top 50,"ROSÉ, Bruno Mars",169917.0,0.783,pop,global,...,2IYQwwgxgOIn7t3iF6ufFD,37i9dQZEVXbMDoHDwVN2tF,5vNRhkKd0yEAg8suGBpjeY,0.939,0.0,149.027,-4.477,0.0283,0.3550,APT.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1740,4.0,41,0.1080,0.590,Meditative Vibes,"Bnxn, taves",133564.0,0.576,ambient,meditative,...,3dctFKCiGPizmylLp0SN0a,3kEpLybbAlVUVimW5Evijb,5odFUgF0MS8DAnvlJpYdUd,0.694,1.0,81.964,-6.530,0.4650,0.0876,"Sincerely, Benson"
731,4.0,41,0.2050,0.841,Arab X,Bnxn,163224.0,0.430,arabic,modern,...,3dctFKCiGPizmylLp0SN0a,3gZgGRvDn9tTUy8n7699pD,7f3xivnGz4HU0UigVxvlEe,0.670,7.0,99.933,-6.006,0.6980,0.1010,"Sincerely, Benson"
721,4.0,41,0.1080,0.590,Arab X,"Bnxn, taves",133564.0,0.576,arabic,modern,...,3dctFKCiGPizmylLp0SN0a,3gZgGRvDn9tTUy8n7699pD,5odFUgF0MS8DAnvlJpYdUd,0.694,1.0,81.964,-6.530,0.4650,0.0876,"Sincerely, Benson"
656,4.0,39,0.3850,0.752,Arab X,Olamide,127819.0,0.811,arabic,modern,...,51mowf1u3WaEYvqalsbP7M,3gZgGRvDn9tTUy8n7699pD,2ciHkdcs9fuSaMn5NhvUNA,0.563,0.0,99.699,-6.185,0.3690,0.4540,Unruly


I'm also a bit unsure of whether to remove only id duplicates or also name/artist duplicates\
I don't want duplicates to distort the distribution of the data.\
But should I also remove a vastly different version (musically speaking) of a song already in the dataset?\
I first want to take a look at the few name/artist duplicates that are no id duplicates

In [29]:
idDupes = songs.duplicated(subset=['track_id'], keep=False)
nameDupes = songs.duplicated(subset=['track_name', 'track_artist'], keep=False)

print(songs.index[idDupes])

Index([ 676,    0,  455,    4,  677,  456,    1,  678,    5,  688,
       ...
       1753,  722, 1778,  693, 1766, 1740,  731,  721,  656, 1771],
      dtype='int64', length=631)
